# 使用 Transformers 进行 Yi 模型推理

欢迎来到这个使用 Hugging Face Transformers 库进行 Yi 模型推理的教程！在这个 Notebook 中,我们将一步步指导你如何使用 Transformers 来加载和运行 Yi-1.5-6B-Chat 模型。这个过程很简单,即使你是新手也能轻松掌握。

## 为什么选择 Transformers?

Hugging Face 的 Transformers 库是一个非常流行的开源 Python 库,它提供了:
- 大量预训练的模型
- 简单易用的 API
- 强大的社区支持

使用 Transformers,你可以轻松地下载、加载和使用各种基于 Transformer 架构的模型,包括我们今天要使用的 Yi 模型。

让我们开始吧!

## 步骤 1: 安装必要的库

首先,我们需要安装 Transformers 库和其他必要的依赖。运行下面的单元格来完成安装:

In [1]:
!pip install transformers>=4.36.2
!pip install gradio>=4.13.0
!pip install torch>=2.0.1,<=2.3.0
!pip install accelerate
!pip install sentencepiece

/bin/bash: line 1: =2.3.0: No such file or directory
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)


这些库的作用如下:
- `transformers`: 用于加载和使用 Yi 模型
- `gradio`: 用于创建简单的 Web 界面(如果需要的话，教程其实没有，如果需要可以参考其它的notebook)
- `torch`: PyTorch 库
- `accelerate`: 用于加速模型加载和推理
- `sentencepiece`: 用于模型的分词处理

安装完成后,我们就可以开始使用模型了!

## 步骤 2: 导入必要的库并加载模型

现在,让我们导入需要的库并加载 Yi-1.5-6B-Chat 模型。

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设置模型路径
model_path = '01-ai/Yi-1.5-6B-Chat'

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # 自动选择可用的设备
    torch_dtype='auto'  # 自动选择合适的数据类型
).eval()  # 将模型设置为评估模式

print("Model loading complete!")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loading complete!


在这段代码中:
- `AutoTokenizer` 用于加载与模型匹配的分词器。
- `AutoModelForCausalLM` 用于加载语言模型。
- `device_map="auto"` 让模型自动选择最佳的设备(CPU 或 GPU)。
- `torch_dtype='auto'` 自动选择合适的数据类型,以优化性能。
- `.eval()` 将模型设置为评估模式,这在进行推理时很重要。

⚠️ 注意: 加载模型可能需要一些时间,具体取决于你的网络速度和电脑性能。

## 步骤 3: 准备输入并进行推理

现在我们的模型已经加载完毕,让我们来尝试进行一次简单的对话!

In [4]:
# 准备对话内容
messages = [
    {"role": "user", "content": "Hi!"}
]

# 将对话转换为模型可以理解的格式
input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')

# 使用模型生成回复
output_ids = model.generate(input_ids.to('cuda'))

# 解码模型的输出
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

print(f"Yi: {response}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Yi: Hello! How can I assist you today?


让我们来解释一下这段代码:

1. 我们首先创建了一个包含用户消息的列表。
2. `apply_chat_template` 方法将我们的消息转换为模型可以理解的格式。
3. `model.generate` 使用转换后的输入生成回复。
4. 最后,我们使用 `tokenizer.decode` 将模型的输出转换回可读的文本。

你可以通过修改 `messages` 列表来尝试不同的对话!

## 进阶: 创建一个简单的对话函数

为了更方便地与模型进行多轮对话,我们可以创建一个简单的函数:

In [6]:
def chat_with_yi(user_input, history=[]):
    # 将新的用户输入添加到历史对话中
    history.append({"role": "user", "content": user_input})

    # 准备输入
    input_ids = tokenizer.apply_chat_template(conversation=history, tokenize=True, add_generation_prompt=True, return_tensors='pt')

    # 生成回复
    output_ids = model.generate(input_ids.to('cuda'), max_new_tokens=100)

    # 解码回复
    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

    # 将模型的回复添加到历史对话中
    history.append({"role": "assistant", "content": response})

    return response, history

# 测试对话函数
history = []
user_inputs = ["Hi!", "Can you tell me a joke?"]

for user_input in user_inputs:
    print(f"user: {user_input}")
    response, history = chat_with_yi(user_input, history)
    print(f"Yi: {response}\n")

user: Hi!
Yi: Hello! How can I assist you today?

user: Can you tell me a joke?
Yi: Sure, here's one for you:

Why don't skeletons fight each other?

They don't have the guts.



这个函数允许你轻松地进行多轮对话,并保持对话的上下文。你可以根据需要继续添加更多的用户输入来测试模型的表现。

## 结语

恭喜你!你已经成功使用 Transformers 库加载并运行了 Yi-1.5-6B-Chat 模型。

本次教程到这里就结束啦!如果你有任何问题,随时查阅 Transformers 的[官方文档](https://huggingface.co/docs/transformers/index)或寻求Yi社区的帮助。